In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 평균-분산 모형 포트폴리오 결합

import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib as mpl
  
folder_dir = '/content/drive/MyDrive/colab'
DATA_FILE = "KRX100재무주가.xlsx"      # 2008년 1월 2일 ~~~
DATA_FILE2 = "KRX100재무주가_원본.xlsx" 

data_wb = pd.ExcelFile(folder_dir + "/" + DATA_FILE)
data_wb2 = pd.ExcelFile(folder_dir + "/" + DATA_FILE2)
         
adj_ts = data_wb.parse("재무", index_col=0)
adj_ts2 = data_wb2.parse("재무",header = [0,1,2], index_col=0)
code = data_wb.parse("코드")
price = data_wb.parse("주가")
price2 = data_wb2.parse("주가", header = [0,1,2], index_col=0)



In [ ]:
!apt install python3-pymysql
import pymysql

In [ ]:

def use_db(hst,un,pw,cs):
  db = pymysql.connect(host=hst,
                     user = un, password=pw,
                     charset=cs)
  cursor = db.cursor()
  return db, cursor
def close_db(db):
  db.commit()
  db.close()

# sql = """insert into STOCKT (`ID`, `Name`, `Per`, `Roe`, `BIS`) values (%s,%s,%s)"""
# for i in 데이터덩이들:
#   cursor.execute(sql,(i[0],i[1],float(dividend[i])))
# close_db(db)

In [ ]:

db,cursor = use_db("jupoza.c7chsemepe5s.ap-northeast-2.rds.amazonaws.com",
                   "admin","jupoza123","utf8")
cursor.execute('USE JUPOZA;')

0

In [ ]:
def calc_ration(past, cur):
    ratio = (cur - past) / np.abs(past)
    ratio *= 100
    return ratio
# 매출 증가율
def revenue_recent_sort(df_name):
    df = pd.ExcelFile(df_name)
    df = df.parse("재무", header=[0, 1, 2], index_col=0)
    total_revenue = df.xs("매출액(천원)", axis=1, level=2)
    total_revenue.columns = total_revenue.columns.droplevel(0)
    total_revenue = total_revenue[-3:-1]
    total_revenue = total_revenue.apply(lambda x: calc_ration(x[0], x[1]), axis=0)
    total_revenue = total_revenue.sort_values(ascending=False).to_dict()
    return total_revenue
# 순이익 증가율
def profit_recent_sort(df_name):
    df = pd.ExcelFile(df_name)
    df = df.parse("재무", header=[0, 1, 2], index_col=0)
    total_profit = df.xs("당기순이익(천원)", axis=1, level=2)
    total_profit.columns = total_profit.columns.droplevel(0)
    total_profit = total_profit[-3:-1]
    total_profit = total_profit.apply(lambda x: calc_ration(x[0], x[1]), axis=0)
    total_profit = total_profit.sort_values(ascending=False).to_dict()
    return total_profit
# 연간 주식 상승률
def stock_anu_rank(df_name):
    df = pd.ExcelFile(df_name)
    df = df.parse("주가", header=[0, 1, 2], index_col=0)
    stock_anu_rate = df.xs("수정주가(원)", axis=1, level=2)
    stock_anu_rate.columns = stock_anu_rate.columns.droplevel(0)
    stock_anu_rate = stock_anu_rate.resample('1Y').first().pct_change()[1:]
    stock_anu_rate = stock_anu_rate.iloc[-1, :].sort_values(ascending=False).to_dict()

    return stock_anu_rate

# 표준편차
def stock_anu_std_rank(df_name):
    df = pd.ExcelFile(df_name)
    df = df.parse("주가", header=[0, 1, 2], index_col=0)
    stock_anu = df.xs("수정주가(원)", axis=1, level=2)
    stock_anu.columns = stock_anu.columns.droplevel(0)

    stock_anu = stock_anu.loc[stock_anu.index > '2021-11-01']
    daily_ret = stock_anu.pct_change()
    stdr = daily_ret.std()
    stdr_rank = stdr.sort_values(ascending=False).to_dict()

    return stdr_rank

In [ ]:
rev = revenue_recent_sort(folder_dir + "/" + DATA_FILE2)
pro = profit_recent_sort(folder_dir + "/" + DATA_FILE2)
stock_rate = stock_anu_rank(folder_dir + "/" + DATA_FILE2)
std = stock_anu_std_rank(folder_dir + "/" + DATA_FILE2)

In [ ]:
display(rev, pro)

In [ ]:
# import math

# sql = """ UPDATE `stock` SET sale = %s, net = %s WHERE stockid = %s"""

# for a in code:
#   net = pro[adj_ts[a][0]]
#   if math.isnan(net):
#     net = 0
#   display(adj_ts[a][0] + " 순이익증가율: "  + str(net)) 
#   revv = rev[adj_ts[a][0]]
#   if math.isnan(revv):
#     revv = 0
#   display(adj_ts[a][0] + " 매출액 증가율: "  + str(revv)) 
#   cursor.execute(sql, (revv, net, int(a[1:])))

In [ ]:
import math

divid = adj_ts2.xs("DPS(보통주,현금)(원)", axis=1, level=2)
df3 = divid.loc["2021-12-31"]

sql = """INSERT INTO `STOCK` (stockid, name, price, per, roe, net, sale, dividend, bis, current, quick, debt, score, rate, std) VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for a in code:
  display(adj_ts[a][0])
  rate1 = round(float(adj_ts[a + ".13"][13]) / float(adj_ts[a + ".8"][13]) * 100, 2)
  if math.isnan(rate1):
    rate1 = 0
  display("자기자본비율: " +  str(rate1))
  rate2 = round(float(adj_ts[a + ".9"][13]) / float(adj_ts[a + ".12"][13]) * 100, 2)
  if math.isnan(rate2):
    rate2 = 0
  display("유동비율: " +  str(rate2))
  rate3 = round(float(adj_ts[a + ".10"][13]) / float(adj_ts[a + ".11"][13]) * 100, 2)
  if math.isnan(rate3):
    rate3 = 0
  display("당좌비율: " +  str(rate3))
  rate4 = round(float(adj_ts[a + ".11"][13]) / float(adj_ts[a + ".13"][13]) * 100, 2)
  if math.isnan(rate4):
    rate4 = 0
  display("부채비율: " +  str(rate4))
  rate5 = round(float(adj_ts[a + ".3"][13]) / (float(adj_ts[a + ".13"][13]) -  float(adj_ts[a + ".11"][13])) * 100, 2)
  if math.isnan(rate5):
    rate5 = 0
  display("ROE: " +  str(rate5))
  # 2963 -> 2021-12-30 주가 
  rate6 = round(float(price[a][2963]) / float(adj_ts[a][13]), 2)
  if math.isnan(rate6):
    rate6 = 0
  display("PER: " +  str(rate6))
  point = 0;
  if rate1 >= 40:
    point += 2
  elif rate1 >= 30:
    point += 1
  if rate2 >= 200:
    point += 2
  elif rate2 >= 150:
    point += 1
  if rate3 >= 150:
    point += 2
  elif rate3 >= 100:
    point += 1
  if rate4 <= 50:
    point += 2
  elif rate4 <= 100:
    point += 1
  display("안정성 점수: " + str(point))
  dividend = round(df3[a][0] / price[a][3166] * 100 , 2)
  if math.isnan(dividend):
    dividend = 0
  display("배당률: " + str(dividend))
  netprofit = round(pro[adj_ts[a][0]] * 100, 2)
  revenue = round(rev[adj_ts[a][0]] * 100, 2)
  if math.isnan(netprofit):
    netprofit = 0
  display("순이익 증가율: " + str(netprofit))
  if math.isnan(revenue):
    revenue = 0
  display("매출 증가율: " + str(revenue))
  stockRate = round(stock_rate[adj_ts[a][0]] * 100, 2)
  if math.isnan(stockRate):
    stockRate = 0
  display("연간 주식 상승율: " + str(stockRate))
  stdd = std[adj_ts[a][0]]
  if math.isnan(stdd):
    stdd = 0
  display("표준편차: " + str(stdd))
  # 배당 df3[a] price2.loc["2022-10-31"]
  # ID, 이름, 주가, per, roe, netprofit, 매출, 배당률, 자기자본비율, 유동비율, 당좌비율, 부채비율, 안정성 점수, 표준편차, 연간주식상승률
  # sql = """INSERT INTO `STOCK` (stockid, name, price, per, roe, net, sale, dividend, bis, current, quick, debt, score, rate) VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
  cursor.execute(sql, (int(a[1:]), adj_ts[a][0], float(price[a][3166]), float(rate6), float(rate5), float(netprofit), float(revenue), float(dividend), float(rate1), float(rate2), float(rate3), float(rate4), float(point), float(stockRate), float(stdd) ))

  

In [ ]:
cursor.execute("COMMIT")

0